In [ ]:
import duckdb
import pandas as pd
# No need to import duckdb_engine
#  jupysql will auto-detect the driver needed based on the connection string!

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

In [ ]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [ ]:
%sql duckdb:///duckdb.db

In [ ]:
%%sql
ATTACH '../data/raw/igem_parts_registry.sqlite' (TYPE sqlite);

In [ ]:
%%sql
SELECT * FROM duckdb_tables();

In [ ]:
%%sql
df_parts << SELECT * FROM igem_parts_registry.parts;

In [ ]:
query = df_parts[df_parts.m_datetime == "2023-09-04 08:01:20"]
query

In [ ]:
df_parts.columns

In [ ]:
condition1 = df_parts.part_type == 'Composite'
condition2 = df_parts.part_type == 'Composite'